In [7]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers import Dense, LSTM,Activation,Dropout

from keras.optimizers import RMSprop

import numpy as np
import random
import sys

Using TensorFlow backend.


In [9]:
import os
data_file =  "./shakespeare_final.txt"
text = open(data_file).read().lower()

print('corpus length:', len(text))
characters = sorted(list(set(text)))
print('total chars:', len(characters))

corpus length: 581432
total chars: 61


In [10]:
char2indices = dict((c, i) for i, c in enumerate(characters))
indices2char = dict((i, c) for i, c in enumerate(characters))

In [11]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 193798


In [12]:
# Converting indices into vectorized format
X = np.zeros((len(sentences), maxlen, len(characters)), dtype=np.bool)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char2indices[char]] = 1
    y[i, char2indices[next_chars[i]]] = 1

In [13]:
#Model Building
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(characters))))

model.add(Dense(len(characters)))

model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

print (model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               97280     
_________________________________________________________________
dense_1 (Dense)              (None, 61)                7869      
_________________________________________________________________
activation_1 (Activation)    (None, 61)                0         
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
# Function to convert prediction into index
def pred_indices(preds, metric=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / metric
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [15]:
# Train & Evaluate the Model
for iteration in range(1, 30):
    print('-' * 40)
    print('Iteration', iteration)
    model.fit(X, y,batch_size=128,epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.7,1.2]:

        print('\n----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(characters)))
            for t, char in enumerate(sentence):
                x[0, t, char2indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = pred_indices(preds, diversity)
            pred_char = indices2char[next_index]

            generated += pred_char
            sentence = sentence[1:] + pred_char

            sys.stdout.write(pred_char)
            sys.stdout.flush()
        print("\nOne combination completed \n")

----------------------------------------
Iteration 1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
193798/193798 [==============================] - 60s 309us/step - loss: 1.8998

----- diversity: 0.2
----- Generating with seed: "k'st not me; 'tis caesar thou defeat'st."
k'st not me; 'tis caesar thou defeat'st.
    the shall the shall the streath the more the chare
    the share the canther that the shall shall
    the streath the love the shall her fortune the comes the shall the preathing in the bear the share
    the shall not ofe the love the will as the there the shall the seart the prease the father shall
    the shall the will a bentresseant the servine the with the buther the shall the love the 
One combination completed 


----- diversity: 0.7
----- Generating with seed: "k'st not me; 'tis caesar thou defeat'st."
k'st not me; 'tis caesar thou defeat'st.
    for have you care, they assary of the greate man's noty be.
  rosalind. and no your that ose siruly shares the

                                                                                                                                                                                                                                                                                                                                                                                                                  
One combination completed 


----- diversity: 0.7
----- Generating with seed: "orn so.
  king. take her by the hand,
  "
orn so.
  king. take her by the hand,
    and you the contime of the sovereare pastes,
    not i had o, since tallame, for we good, mercand,
    bring chass with his fortune, and, and,
    and is i when i past of my etext you. if i do not do to cauce the sister be liones,
  for downlossion, and prove to the way,
  and to his is the i's such a did his lose and
    hast the nough happiness be o beautily, and,
                             
One combination completed 


----- 

    by my fortunaus, ill, honest than her be shouldst the world thee,
  har of the even resencet, their bl
One combination completed 


----- diversity: 1.2
----- Generating with seed: "t these woods
    more free from peril t"
t these woods
    more free from peril thus herm'dt foon't a b'ty choest?'
 [stry bee him marmoned to wear.
  cleopatra.  way not lionsof.
  lafeu. to poce not pompey, the wedble
    theigh mariute, but as no, though
    like himbelst, warn mender her charles,
    i troshy atter trusp to live not flof metviredly, and how thy supster an.
  parolles. o must be muchk!
       

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


     'sary eyes, caesar appature.
  orlando. in you now nor my mo
One combination completed 

----------------------------------------
Iteration 10
Epoch 1/1
193798/193798 [==============================] - 66s 341us/step - loss: 1.3375

----- diversity: 0.2
----- Generating with seed: "ghs of horse to tell of her approach
   "
ghs of horse to tell of her approach
    the string'd the string'd the man in the string,
    the string'd the self thee well the self the service; but i have for his world of the soldiers of the soldiers of the world of the great service the string'd the man.
    when i shall the self the forest word a fortunes many the string'd the self the self the man.
                                                                                 
One combination completed 


----- diversity: 0.7
----- Generating with seed: "ghs of horse to tell of her approach
   "
ghs of horse to tell of her approach
    should stay be brake og't that women thee.
  charmian. as you me an

    into th' aby to my awally enobarbus that i shall death.
    i eat alexand my sannbus.
  soldier. the lardous the farsh i am my poor on you
    ass him of a she worther the end whereon,
    that for a part, i am no more paod you and his shall cannot gany;
    i shall thy end that i have a down bing.
    a cheev'd on the little perforly and like,
    that i' my remov'd of heaven'd, why a cases,
    whose which 
One combination completed 


----- diversity: 1.2
----- Generating with seed: "bs and shot their fires
    into th' aby"
bs and shot their fires
    into th' aby urtly. of his enter
    thy is goest th' advis their of coneded
    foungh imbemp, dinne sasious. napy you chathon bart
  samband and
    wh fimbolulith him. list, so horn that bene.
  ear that good troed theit hoply, from they haok red rather
    tumbard our name' and cithe lovery's fard.


                       12
    unlianls?
  thou dresen, make's mervhan that cast detory;
    true, 
       
One combination compl

  soorrau. let sioulawary you socleme;
    ill? the sarierve of his carveneram.
  rosalind. if so, egypt's  nofelf that hav
One combination completed 

----------------------------------------
Iteration 19
Epoch 1/1
193798/193798 [==============================] - 63s 325us/step - loss: 1.2930

----- diversity: 0.2
----- Generating with seed: "kes thy love more strong,
    to love th"
kes thy love more strong,
    to love the sure with me the sure the sure,
    the countery the sure of the forest beard.
    i will for your man the death of the sure with the soldier
    the despits the forest with the forest beard,
    the seafing that which i shall see the best beard.
    the sure more to the sure beard in the sure the sure,
    and the sure the forest beauty beauty sorrow,
    the forest of the sure with me. i have 
One combination completed 


----- diversity: 0.7
----- Generating with seed: "kes thy love more strong,
    to love th"
kes thy love more strong,
    to love this desires

ius. i'll humbly signify what in his name; but i do thank her be not hate,
       so are not better i with declay,
    though i to serves thou dare hours?
  i will prove i must to the earn the have devil
    to love a brother been suw provnisg him she hath for letter
    thou art and near that my lifes of the coursel,
    are with beauty rath that as you must to good serve much thou not well.
    is it from his so keeve and shall we do 
One combination completed 


----- diversity: 1.2
----- Generating with seed: "ius. i'll humbly signify what in his nam"
ius. i'll humbly signify what in his name, guardship
mance o' th' cioon doveble your beorstand,
    though by honour and roman. as now [awavewall combend

             exeunt


act ri's celio-but

  menas.     emptiding together
    is turt into poor is pride bid her.
  lambod from my freelse, het court?
  i sauder of my low would shall draw not faig?
  endushida. becareals, wsill to imagint my father.
  amiens and own thereing.
  dou

   upon upleases with owing you be unties thee, so her place's life sick
    be lidsons, go onest worthied ashely?
  rosalind. we recking of high very 
One combination completed 

----------------------------------------
Iteration 28
Epoch 1/1
193798/193798 [==============================] - 62s 318us/step - loss: 1.2670

----- diversity: 0.2
----- Generating with seed: "ndo, that tripp'd up the wrestler's heel"
ndo, that tripp'd up the wrestler's heels,
    and that where is the strange than they will this than the sure,
    and that he world that he will be the for the true and so the sure and for the sure,
    the for that hath the married to be thine,
    that have i have that where is not that hath the horse,
    and thou wilt be not that the world between the sure,
  and that which i am so bright to friends,
    and and that i would have 
One combination completed 


----- diversity: 0.7
----- Generating with seed: "ndo, that tripp'd up the wrestler's heel"
ndo, that tripp'd up t